In [84]:
from sys import path
import json
import codecs
import json
import csv
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import glob
path.append(r"/opt/configs/dev")

from config import MONGO_DB, MONGO_HOST, MONGO_PASSWD, MONGO_USER, MONGO_PORT


from pymongo import MongoClient


cli = MongoClient(f"mongodb://{MONGO_USER}:{MONGO_PASSWD}@{MONGO_HOST}:{MONGO_PORT}")
db = cli['leaked']

# TODO 建索引
users_col = db['persons']
users_col.create_index("name")
users_col.create_index("mobile")
users_col.create_index("id_no")

police_case_col = db['police_cases']
police_case_col.create_index("case_id")

'case_id_1'

# 202208 上海公安 10 亿数据泄露
其中的 75w 示例数据

数据格式很乱，不是标准 JSON。
需要逐个文件处理，用正则提取有效信息。

## address_merge_with_mobile_data.json

```
{"_source":{"APP_TYPE":"1229999","ATTR_IDENT_LABEL":"07","ATTR_IDENT_LABEL_DETL":"{"LABEL_DETL":[{"07":"shga_wa.ods_nb_tab_goods"}]}","BIG_SOURCE":"WA","COUNT":"3","DATA_SOURCE":"115","DETAIL":"{"nameinfo":[{"name":"刘婵女士"}]}","FIRST_TIME":"1562126476","IDENTITY_TYPE":"mobile","IDENTITY_VALUE":"18627855112","LAST_TIME":"1562557677","MRG_ID":"cda7c53c06248628cd33777dff71d9a3","SRC_ADDRESS":"智慧e谷大楼(5楼)","SRC_ID":"cda7c53c06248628cd33777dff71d9a3","TABLE_SOURCE":"shga_wa.ods_nb_tab_goods"},"_type":"a","sort":[33150]}"""
```

In [ ]:
import re

raw = """{"_source":{"APP_TYPE":"1229999","ATTR_IDENT_LABEL":"07","ATTR_IDENT_LABEL_DETL":"{"LABEL_DETL":[{"07":"shga_wa.ods_nb_tab_goods"}]}","BIG_SOURCE":"WA","COUNT":"3","DATA_SOURCE":"115","DETAIL":"{"nameinfo":[{"name":"刘婵女士"}]}","FIRST_TIME":"1562126476","IDENTITY_TYPE":"mobile","IDENTITY_VALUE":"18627855112","LAST_TIME":"1562557677","MRG_ID":"cda7c53c06248628cd33777dff71d9a3","SRC_ADDRESS":"智慧e谷大楼(5楼)","SRC_ID":"cda7c53c06248628cd33777dff71d9a3","TABLE_SOURCE":"shga_wa.ods_nb_tab_goods"},"_type":"a","sort":[33150]}"""
regex = r"\"name\":\"(?P<name>[^\"]+)\".*?\"IDENTITY_TYPE\":\"(?P<type>[^\"]+)\".*?\"IDENTITY_VALUE\":\"(?P<val>[^\"]+)\".*?\"SRC_ADDRESS\":\"(?P<addr>[^\"]+)\".*?"

addr_regex = re.compile(regex)
addr_regex.findall(raw)


In [ ]:
import codecs

# {"_source":{"APP_TYPE":"1229999","ATTR_IDENT_LABEL":"07","ATTR_IDENT_LABEL_DETL":"{"LABEL_DETL":[{"07":"shga_wa.ods_nb_tab_goods"}]}","BIG_SOURCE":"WA","COUNT":"3","DATA_SOURCE":"115","DETAIL":"{"nameinfo":[{"name":"刘婵女士"}]}","FIRST_TIME":"1562126476","IDENTITY_TYPE":"mobile","IDENTITY_VALUE":"18627855112","LAST_TIME":"1562557677","MRG_ID":"cda7c53c06248628cd33777dff71d9a3","SRC_ADDRESS":"智慧e谷大楼(5楼)","SRC_ID":"cda7c53c06248628cd33777dff71d9a3","TABLE_SOURCE":"shga_wa.ods_nb_tab_goods"},"_type":"a","sort":[33150]}


i = 0
with codecs.open(r"/home/laisky/Downloads/shga/address_merge_with_mobile_data.json", 'r', 'utf8') as fp:
    for line in fp:
        matched = addr_regex.findall(line)
        if len(matched) != 1 or matched[0][1].lower() != "mobile":
            continue
            
        docu = {
            "name": matched[0][0],
            "mobile": matched[0][2],
            "address": matched[0][3],
        }
        users_col.update_one(
            filter={"mobile": docu['mobile']},
            update={"$set": docu},
            upsert=True,
        )
        i += 1
        if i % 1000 == 0:
            print(f"inserted {i}")
            
print(f"finished {i}")

## case_data_index.json

```
{"_id":"AW7l981GkKgaptTtyNFG","_index":"nb_theme_address_case_dwd-total","_score":null,"_source":{"ADDR_DETL":"{"CASE":{"BRIEF_CASE":"双方已 自行解决","CASE_TYPE":null,"CASE_STATE":null},"ORGANIZER_POLICE_TYPE":null,"ORGANIZER":{"ORGANIZER_NAME":null,"ORGANIZER_AREA":null},"TIME":{"ACCEPTANCE_TIME":null,"REGISTER_TIME":null,"CLOSING_TIME":null,"PUNISH_TIME":null,"CASE_TIME":null,"REPORT_TIME":null},"ACCEPTANCE_UNIT_JSON":{"ACCEPTANCE_UNIT_NUMBER":null,"ACCEPTANCE_UNIT_NAME":null},"TABLE_SOURCE":"shga_dwd.base_wsba_hx_a_jbxx_df","CASE_ADDRESS":"上海市浦东新区沪南路红星美凯龙"}","ADDR_TYPE":"01","CASE_NUMBER":"B3101154600002012020274","CITY_L2":"上海市","CONFIDENCE":"4","COUNTY_L3":"浦东新区","GEOHASH7":"wtw3ybx","GEOHASH8":"wtw3ybx1","GEOHASH9":"wtw3ybx1y","LATITUDE":"31.248198","LOC_SOURCE":"0","LONGITUDE":"121.595571","MCS_ID":"367fcf62b5add3056b0fe579310e8f51","POI_L8":"红星美凯龙","PROVINCE_L1":"上海市","ROAD_L5":"沪南路","STD_ADDRESS":"上海市上海市浦东新区沪南路红星美 凯龙"},"_type":"a","sort":[26091]}
{"_id":"AW5jx4iFcR021UzX44dg","_index":"nb_theme_address_case_dwd-total","_score":null,"_source":{"ADDR_DETL":"{"CASE":{"BRIEF_CASE":"2017年11月21日15时许,我所民警接群众匿名举报称:本市徐汇区嘉善路171弄11号棋牌室内有人以打麻将方式进行赌博。","CASE_TYPE":null,"CASE_STATE":null},"ORGANIZER_POLICE_TYPE":null,"ORGANIZER":{"ORGANIZER_NAME":null,"ORGANIZER_AREA":null},"TIME":{"ACCEPTANCE_TIME":null,"REGISTER_TIME":null,"CLOSING_TIME":null,"PUNISH_TIME":null,"CASE_TIME":null,"REPORT_TIME":null},"ACCEPTANCE_UNIT_JSON":{"ACCEPTANCE_UNIT_NUMBER":null,"ACCEPTANCE_UNIT_NAME":null},"TABLE_SOURCE":"shga_dwd.base_wsba_hx_a_ajjbqk_df","CASE_ADDRESS":"上海市徐汇区嘉善路171弄11号"}","ADDR_TYPE":"02","BLOCK_L4":"天 平路街道","BUILDING_L9":"11号","CASE_NAME":"嘉善路171弄11号赌博案","CASE_NUMBER":"C3101044300002017110223","CITY_L2":"上海市","CONFIDENCE":"5","COUNTY_L3":"徐汇区","GEOHASH7":"wtw3eck","GEOHASH8":"wtw3eckt","GEOHASH9":"wtw3eckt5","LATITUDE":"31.208933","LOC_SOURCE":"0","LONGITUDE":"121.460201","MCS_ID":"b2856846825d9c82b2f164b5f107c9a9","NONG_L6":"171弄","PROVINCE_L1":"上海市","ROAD_L5":"嘉善路","STD_ADDRESS":"上海市上 海市徐汇区天平路街道嘉善路171弄11号"},"_type":"a","sort":[26091]}

```

In [ ]:
raw = """
{"_id":"AW7l981GkKgaptTtyNFG","_index":"nb_theme_address_case_dwd-total","_score":null,"_source":{"ADDR_DETL":"{"CASE":{"BRIEF_CASE":"双方已 自行解决","CASE_TYPE":null,"CASE_STATE":null},"ORGANIZER_POLICE_TYPE":null,"ORGANIZER":{"ORGANIZER_NAME":null,"ORGANIZER_AREA":null},"TIME":{"ACCEPTANCE_TIME":null,"REGISTER_TIME":null,"CLOSING_TIME":null,"PUNISH_TIME":null,"CASE_TIME":null,"REPORT_TIME":null},"ACCEPTANCE_UNIT_JSON":{"ACCEPTANCE_UNIT_NUMBER":null,"ACCEPTANCE_UNIT_NAME":null},"TABLE_SOURCE":"shga_dwd.base_wsba_hx_a_jbxx_df","CASE_ADDRESS":"上海市浦东新区沪南路红星美凯龙"}","ADDR_TYPE":"01","CASE_NUMBER":"B3101154600002012020274","CITY_L2":"上海市","CONFIDENCE":"4","COUNTY_L3":"浦东新区","GEOHASH7":"wtw3ybx","GEOHASH8":"wtw3ybx1","GEOHASH9":"wtw3ybx1y","LATITUDE":"31.248198","LOC_SOURCE":"0","LONGITUDE":"121.595571","MCS_ID":"367fcf62b5add3056b0fe579310e8f51","POI_L8":"红星美凯龙","PROVINCE_L1":"上海市","ROAD_L5":"沪南路","STD_ADDRESS":"上海市上海市浦东新区沪南路红星美 凯龙"},"_type":"a","sort":[26091]}
{"_id":"AW5jx4iFcR021UzX44dg","_index":"nb_theme_address_case_dwd-total","_score":null,"_source":{"ADDR_DETL":"{"CASE":{"BRIEF_CASE":"2017年11月21日15时许,我所民警接群众匿名举报称:本市徐汇区嘉善路171弄11号棋牌室内有人以打麻将方式进行赌博。","CASE_TYPE":null,"CASE_STATE":null},"ORGANIZER_POLICE_TYPE":null,"ORGANIZER":{"ORGANIZER_NAME":null,"ORGANIZER_AREA":null},"TIME":{"ACCEPTANCE_TIME":null,"REGISTER_TIME":null,"CLOSING_TIME":null,"PUNISH_TIME":null,"CASE_TIME":null,"REPORT_TIME":null},"ACCEPTANCE_UNIT_JSON":{"ACCEPTANCE_UNIT_NUMBER":null,"ACCEPTANCE_UNIT_NAME":null},"TABLE_SOURCE":"shga_dwd.base_wsba_hx_a_ajjbqk_df","CASE_ADDRESS":"上海市徐汇区嘉善路171弄11号"}","ADDR_TYPE":"02","BLOCK_L4":"天 平路街道","BUILDING_L9":"11号","CASE_NAME":"嘉善路171弄11号赌博案","CASE_NUMBER":"C3101044300002017110223","CITY_L2":"上海市","CONFIDENCE":"5","COUNTY_L3":"徐汇区","GEOHASH7":"wtw3eck","GEOHASH8":"wtw3eckt","GEOHASH9":"wtw3eckt5","LATITUDE":"31.208933","LOC_SOURCE":"0","LONGITUDE":"121.460201","MCS_ID":"b2856846825d9c82b2f164b5f107c9a9","NONG_L6":"171弄","PROVINCE_L1":"上海市","ROAD_L5":"嘉善路","STD_ADDRESS":"上海市上 海市徐汇区天平路街道嘉善路171弄11号"},"_type":"a","sort":[26091]}
"""
regex = r"BRIEF_CASE\":\"([^\"]+)\".*CASE_NUMBER\":\"([^\"]+)\".*CITY_L2\":\"([^\"]+)\".*COUNTY_L3\":\"([^\"]+)\".*LATITUDE\":\"([^\"]+)\".*LONGITUDE\":\"([^\"]+)\".*STD_ADDRESS\":\"([^\"]+)\""

case_regex = re.compile(regex)
case_regex.findall(raw)

In [ ]:
i = 0
with codecs.open(r"/home/laisky/Downloads/shga/case_data_index.json", 'r', 'utf8') as fp:
    for line in fp:
        matched = case_regex.findall(line)
        if len(matched) != 1:
            continue
            
        docu = {
            "brief": matched[0][0].strip(),
            "case_id": matched[0][1].strip(),
            "city": matched[0][2].strip(),
            "county": matched[0][3].strip(),
            "latitude": matched[0][4].strip(),
            "longitude": matched[0][5].strip(),
            "address": matched[0][6].strip(),
        }
        
#         print(docu)
#         break
        
        police_case_col.update_one(
            filter={"case_id": docu['case_id']},
            update={"$set": docu},
            upsert=True,
        )
        i += 1
        if i % 1000 == 0:
            print(f"inserted {i}")
            
print(f"finished {i}")

## person_info.json

```
{"_id":"AXtamfr9mkHrMSbQCSme","_index":"person_address_label_info_master","_score":null,"_source":{"AGE":6,"BIRTHDAY":"2015","BPLACE":"浙江省衢州市柯城区","HHPLACE":"浙江省衢州市柯城区九华乡上铺村徐莫39号","IDNO":"330802201501207944","IDTYPE":"01","PHOTO":"{"身份证":["http://oss-cn-shanghai-shga-d01-a.ops.ga.sh/shga-ryzp/CSJ/ZHEJIANG_CZRK_ZP/59578841_330802.jpg"]}","QUERY_STRING":"  浙江省衢州市柯城区  浙江省衢州市柯城区九华乡上铺村徐莫39号 6 15 2015 ","RNAME":"危婧彤","SEX":"女"},"_type":"a","sort":[6754953]}
{"_id":"AXtamMSkxRtHEfZelWKn","_index":"person_address_label_info_master","_score":null,"_source":{"AGE":43,"BIRTHDAY":"1978","BPLACE":"湖北省十堰市张湾区","HEIGHT":"160","IDNO":"420303197812271726","IDTYPE":"01","QUERY_STRING":"  湖北省十堰市张湾区   43 78 1978 ","RNAME":"王立华","SEX":"女"},"_type":"a","sort":[6754953]}

```

In [ ]:
raw = """{"_id":"AXtamfr9mkHrMSbQCSme","_index":"person_address_label_info_master","_score":null,"_source":{"AGE":6,"BIRTHDAY":"2015","BPLACE":"浙江省衢州市柯城区","HHPLACE":"浙江省衢州市柯城区九华乡上铺村徐莫39号","IDNO":"330802201501207944","IDTYPE":"01","PHOTO":"{"身份证":["http://oss-cn-shanghai-shga-d01-a.ops.ga.sh/shga-ryzp/CSJ/ZHEJIANG_CZRK_ZP/59578841_330802.jpg"]}","QUERY_STRING":"  浙江省衢州市柯城区  浙江省衢州市柯城区九华乡上铺村徐莫39号 6 15 2015 ","RNAME":"危婧彤","SEX":"女"},"_type":"a","sort":[6754953]}
"""
regex = r"\"IDNO\":\"([^\"]+)\".*?\"QUERY_STRING\":\"([^\"]+)\".*?\"RNAME\":\"([^\"]+)\".*?\"SEX\":\"([^\"]+)\".*?"

person_regex = re.compile(regex)
person_regex.findall(raw)


# json.loads(raw)

In [ ]:
i = 0
with codecs.open(r"/home/laisky/Downloads/shga/person_info.json", 'r', 'utf8') as fp:
    for line in fp:
        matched = person_regex.findall(line)
        if len(matched) != 1:
            continue
            
        docu = {
            "name": matched[0][2].strip(),
            "sex": matched[0][3],
            "address": matched[0][1].strip(),
            "id_no": matched[0][0].strip(),
        }
        
#         print(docu)
#         break
        
        users_col.update_one(
            filter={"id_no": docu['id_no']},
            update={"$set": docu},
            upsert=True,
        )
        i += 1
        if i % 1000 == 0:
            print(f"inserted {i}")
            
print(f"finished {i}")

# 电商泄露

In [ ]:
def get_csv_val(docu, key):
    val = docu.get(key, "")
    if val == "NULL":
        return ""
    
    return val.strip()

with codecs.open(r"/home/laisky/Downloads/user.csv", 'r', 'utf8') as fp:
    reader = csv.DictReader(fp)
    for i, row in enumerate(reader):
        if not get_csv_val(row, "mobile"):
            continue
        
        docu = {
            "name": get_csv_val(row, "realname") or get_csv_val(row, "username"),
            "address": f"{get_csv_val(row, 'pprovince')}{get_csv_val(row, 'ccity')}{get_csv_val(row, 'aarea')}{get_csv_val(row, 'address')}",
            "mobile": get_csv_val(row, "mobile"),
            "email": get_csv_val(row, "email"),
        }
#         print(docu)
        
#         if i > 5:
#             break
            
        # 这份数据的可信度不高，不要覆盖已有字段
        old_docu = users_col.find_one({"mobile": docu['mobile']}) or {}
        docu.update(old_docu)
#         print(docu)
#         if i > 5:
#             break
        users_col.update_one(
            filter={"mobile": docu['mobile']},
            update={"$set": docu},
            upsert=True,
        )
        i += 1
        if i % 1000 == 0:
            print(f"inserted {i}")

# qb-prod

把 caigen100 的数据搬运过来

In [ ]:
caigen_db = cli['qb-prod']
caigen_users = caigen_db['aliusers']
caigen_orders = caigen_db['raworders']

In [ ]:
# 搬运 aliusers

i = 0
for user in caigen_users.find({}, max_time_ms=0):
    docu = {
        "name": user['recipient'],
        "nickname": [user['username']],
        "mobile": user['mobile'], 
    }
    
    old_docu = users_col.find_one({"mobile": docu['mobile']}) or {}
    docu.update(old_docu)
#     print(docu)
#     if i > 5:
#         break
    users_col.update_one(
        filter={"mobile": docu['mobile']},
        update={"$set": docu},
        upsert=True,
    )
    i += 1
    if i % 1000 == 0:
        print(f"inserted {i}")
            
print(f"finished {i}")

In [ ]:
# 更新用户 nickname
# 将字符串转换为 list

for user in users_col.find({'nickname': {'$exists': True}}, max_time_ms=0):
    if isinstance(user['nickname'], list):
        continue
        
    user['nickname'] = [user['nickname']]
#     print(user)
#     break
    users_col.update_one(
        filter={"_id": user['_id']},
        update={"$set": {"nickname": user["nickname"]}}
    )

In [ ]:
# 搬运 raworders

i = 0
for user in caigen_orders.find({}, max_time_ms=0):
    if not user.get('收货人姓名') or \
        not user.get('联系手机'):
        continue
        
    docu = {
        "name": user['收货人姓名'],
        "nickname": user['买家会员名'],
        "mobile": user['联系手机'], 
        "address": user['收货地址'],
        'email': user['买家支付宝账号'],
    }
    
#     print(docu)
    old_docu = users_col.find_one({"mobile": docu['mobile']}) or {}
    old_docu['nickname'] = (old_docu.get('nickname', []) or [])
    old_docu['nickname'].append(docu['nickname'])
    docu.update(old_docu)
#     print(docu)
#     if i > 5:
#         break
    users_col.update_one(
        filter={"mobile": docu['mobile']},
        update={"$set": docu},
        upsert=True,
    )
    i += 1
    if i % 1000 == 0:
        print(f"inserted {i}")
            
print(f"finished {i}")

# 志愿者数据

```json
{
    "addtype": 2,
    "birthYear": "1994",
    "cardnum": "",
    "cardtype": 0,
    "city": "147608411235737ae98eba11e682992c56dcba85b9",
    "city_name": "南昌市",
    "commonwealscore": 0,
    "county": "",
    "county_name": null,
    "createtime": "2016-11-03T19:25:09.000Z",
    "createuser": "147217442364212d442628e12439487b51632dd47407e",
    "creditduration": 0,
    "declaration": "",
    "depts": [
        {
            "deptid": "1477028179659e891b8862de740baa7a91f272a54f639",
            "paths": [
                {
                    "codepath": "000000,360000,360099",
                    "path": "1474445511144f5d954255927464a899c50cb7ed45e7a,14760841123571a6138eba11e682992c56dcba85b9,1477028179659e891b8862de740baa7a91f272a54f639"
                }
            ]
        }
    ],
    "headimage": null,
    "hisduration": 0,
    "idcard": "360403199410171229",
    "idcardEncryption": "7TdBxOu/G358N6fwy9ryoqLaUhpHhvYs4cTo2QMZkpw=",
    "idcardtype": 1,
    "iscansearch": 1,
    "iscertification": 1,
    "label": [],
    "loginname": null,
    "mphone": "",
    "name": "赵丹",
    "nation_name": null,
    "nickname": "",
    "origincity_name": null,
    "origincounty_name": null,
    "originprovince_name": null,
    "password": "27-93-21691224325-70-7930-750-",
    "political": "1474515864045b72536e3bd8447c885977a2a565c6640",
    "political_name": "群众",
    "province": "14760841123571a6138eba11e682992c56dcba85b9",
    "province_name": "江西省",
    "regcity_name": "南昌市",
    "regcounty_name": null,
    "regprovince_name": "江西省",
    "regsourcecity": null,
    "regtype": null,
    "sex": "2",
    "source": "zyh20161125013601",
    "star": null,
    "status": 1,
    "trainduration": 0,
    "updatetime": "2016-11-03T19:25:09.000Z",
    "vcode": "360403001007440041",
    "volunteerdeptlist": null,
    "volunteerextendentity": null
}

```

In [ ]:
i = 0
with codecs.open(r"/home/laisky/Downloads/zyh_sample.json", 'r', 'utf8') as fp:
    for i, row in enumerate(fp):
        row = json.loads(row)
        if not row.get('name').strip():
            continue
        
        docu = {
            "name": row['name'].strip(),
            "address": f"{row['province_name']}{row['regcity_name']}".strip(),
            "id_no": row['idcard'].strip(),
        }
            
        old_docu = users_col.find_one({"id_no": docu['id_no']}) or {}
        if old_docu.get('address', "") > docu['address']:
            docu['address'] = old_docu['address']
            
#         print(docu)
#         if i > 5:
#             break
            
        users_col.update_one(
            filter={"id_no": docu['id_no']},
            update={"$set": docu},
            upsert=True,
        )
        i += 1
        if i % 1000 == 0:
            print(f"inserted {i}")

# 2000w 开房数据

In [57]:
glob.glob(r"/home/laisky/Downloads/2000/*.csv")

['/home/laisky/Downloads/2000/1400W-1600W.csv',
 '/home/laisky/Downloads/2000/800W-1000W.csv',
 '/home/laisky/Downloads/2000/1-200W.csv',
 '/home/laisky/Downloads/2000/600W-800W.csv',
 '/home/laisky/Downloads/2000/最后5000.csv',
 '/home/laisky/Downloads/2000/1000W-1200W.csv',
 '/home/laisky/Downloads/2000/1200W-1400W.csv',
 '/home/laisky/Downloads/2000/1800w-2000w.csv',
 '/home/laisky/Downloads/2000/200W-400W.csv',
 '/home/laisky/Downloads/2000/1600w-1800w.csv',
 '/home/laisky/Downloads/2000/400W-600W.csv']

In [72]:
a = {"a": 123}
a.update({"a": None})
a

{'a': None}

In [ ]:
def get_csv_val(docu, key):
    val = docu.get(key, "")
    if isinstance(val, str):
        val = val.strip()
        if val.lower() == "null":
            return 
        
    return str(val) or None

def clean_dict_none(docu):
    for key, val in list(docu.items()):
        if not val or \
         (isinstance(val, str) and (not val.strip() or val.strip().lower() == 'none')):
            del docu[key]
        
            

def load_kaifang(file):
    i = 0
    with codecs.open(file, 'r', 'utf8') as fp:
        reader = csv.DictReader(fp)
        for row in reader:

            docu = {
                "name": get_csv_val(row, "﻿Name"),
                "address": get_csv_val(row, 'Address'),
                "mobile": get_csv_val(row, "Mobile"),
                "email": get_csv_val(row, "EMail"),
                "sex": get_csv_val(row, "Gender"),
                "id_no": get_csv_val(row, "CtfId"),
            }
            clean_dict_none(docu)
#             print(docu)
#             return

            if docu.get('id_no'):
                users_col.update_one(
                    filter={"id_no": docu['id_no']},
                    update={"$set": docu},
                    upsert=True,
                )
            elif docu.get('mobile'):
                users_col.update_one(
                    filter={"mobile": docu['mobile']},
                    update={"$set": docu},
                    upsert=True,
                )
            elif docu.get('email'):
                users_col.update_one(
                    filter={"email": docu['email']},
                    update={"$set": docu},
                    upsert=True,
                )
            else:
                print(docu)
                continue

            i += 1
            if i % 10000 == 0:
                print(f"inserted {i}")
            
    return i
                

fs = []
with ThreadPoolExecutor(max_workers=20) as executor:    
    for file in glob.glob(r"/home/laisky/Downloads/2000/*.csv"):
        fs.append(executor.submit(load_kaifang, file))
        
list(map(lambda f: f.result(), fs))

{'name': '徐缨'}
{'name': '0755-33665128'}
{'name': 'jpzhao@chinapaint.com.cn'}
inserted 10000
inserted 10000
inserted 10000
inserted 10000
inserted 10000
inserted 10000
inserted 10000
inserted 10000
inserted 10000
inserted 10000
inserted 10000
inserted 20000
inserted 20000
inserted 20000
inserted 20000
inserted 20000
inserted 20000
inserted 20000
inserted 20000
inserted 20000
inserted 20000
inserted 20000
inserted 30000
inserted 30000
inserted 30000
inserted 30000
inserted 30000
inserted 30000
inserted 30000
inserted 30000
inserted 30000
inserted 30000
inserted 30000
inserted 40000
inserted 40000
inserted 40000
inserted 40000
inserted 40000
inserted 40000
inserted 40000
inserted 40000
inserted 40000
inserted 40000
inserted 40000
{'name': '太大集体户'}
inserted 50000
inserted 50000
inserted 50000
inserted 50000
inserted 50000
inserted 50000
inserted 50000
inserted 50000
inserted 50000
inserted 50000
inserted 50000
inserted 60000
inserted 60000
inserted 60000
inserted 60000
inserted 60000
inse